# Parte de Optimización

## Cargamos los datos
En primer luegar, cargamos los datos y los hacemos manejables para la primera aproximacion, transformaremos los dataset obtenidos en la parte anterior en matrices y arrays.

Del archivo "final_data.csv" leemos los consumos de 4 usuarios, ya tratados en la seccion anterior.
Del archivo "generacion.csv" leemos la generacion para 4 usuario.

A prior tenemos 24*30 = 720 columnas, 24 horas y 30 dias, sin embargo, eliminaremos las columnas que tengan generación nula. No tiene sentido optimizar los coeficientes para las horas que no hay sol, en estos casos, los coeficientes seran 0. Con esto, en mi ejemplo, paso de 720 columnas a 455.

En este notebook se utilizan 3 metodos. Primero usando un metodo complejo de optimizacion de funciones no lineales, de la libreria de minimize, en concreto, SLSQP.
Luego, se usa una aproximacion rapida, que calcula los coeficientes en funcion de la hora exclusivamente, segun el orden cronologico de los datos.
En tercer luegar, una modificación del metodo anterior, la idea aquí es que se ordenan los consumos por orden de desperdicio. Así, se empieza a optimizar los coeficientes por los que mas energia vierten. De esta forma, se consigue reducir el vertido de energia.

Luego se explica cada modelo con mas detalle.

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import time

# Leo el archivo final_data.csv y lo guardo en la matriz E
E = pd.read_csv('final_data.csv', index_col=0)

# Leo el archivo de generacion de energia, lo redondeo y paso a kwh. Lo almaceno en total_e
generacion = pd.read_csv('generacion.csv')
generacion = generacion.iloc[:,1]
gen_array = generacion.values
total_e = gen_array
total_e = [round(i/1000, 2) for i in total_e]
total_e = np.array(total_e)

indices_nulos = np.where(total_e == 0)[0] # estos indices serviran para reconstruir luego la matriz de coeficientes, sabiendo que los indices eliminados tendran 0 en la matriz de coeficientes
total_e_no_nulo = np.delete(total_e, indices_nulos)

E_no_nulo = E.drop(E.columns[indices_nulos], axis=1)
print("Las dimensiones de la matriz son: ", E_no_nulo.shape)

n_rows, n_cols = E_no_nulo.shape
nTotHour = total_e_no_nulo.shape[0]
print("  En total hay ", nTotHour, " horas (columnas)")
print("  En total hay ", n_rows, " usuarios (filas)")
nUser = 4

Las dimensiones de la matriz son:  (4, 455)
  En total hay  455  horas (columnas)
  En total hay  4  usuarios (filas)


Ahora uso la libreria de minimize para optimizar el problema. Se usa el metodo SLSQP. Este metodo usa aproximacion del jacobiano de la funcion, para ir en direccion contraria al gradiante. La complejidad reside en el tamaño de los datos y en que la función no es lineal. La función a minimizar es el vertido de energia a la red:

$$ Min \sum_{j=1}^{nCols}\sum_{i=1}^{n} \max(0, c_{ij} \cdot {totalEnergy_j} - E_{ij})$$

Con las restricciones:
$$\sum_{i=1}^n c_{ij} =1,  \forall j \in nCols$$
$$\sum_{j=1}^{nCols} E[i, j] = \frac{nCols}{nUser},  \forall i \in nUser$$

In [4]:
# Usando la funcion minimize de scipy.optimize, minimizamos la funcion objetivo
n_rows, n_cols = E_no_nulo.shape
nUser = 4

def objective_function(c):
    c_matrix = c.reshape((n_rows, n_cols))
    total = 0
    for j in range(n_cols):
        for i in range(n_rows):
            total += max(0, c_matrix[i, j] * total_e_no_nulo[j] - E_no_nulo.iloc[i, j])
    return total

def equality_constraint1(c):
    return np.sum(c.reshape((n_rows, n_cols)), axis=0) - 1
def equality_constraint2(c):
    return np.sum(c.reshape((n_rows, n_cols)), axis=1) - n_cols/n_rows

bounds = [(0, 1)] * (n_rows * n_cols)

initial_guess = np.full(n_rows * n_cols, 0.25)


constraints = ({'type': 'eq', 'fun': equality_constraint1},
               {'type': 'eq', 'fun': equality_constraint2})

seg = 0

def callback_function(xk):
    global seg
    if (callback_function.iteration == 0):
        # guardo la hora de inicio
        seg = time.time()
        
    print("Iteración:", callback_function.iteration)
    print("Variables de decisión:", xk)
    print("Valor de la función objetivo:", objective_function(xk))
    print("--------------------------")
    callback_function.iteration += 1

callback_function.iteration = 0

# carga la variable res si existe   
res = minimize(objective_function, initial_guess, bounds=bounds, constraints=constraints, method='SLSQP', callback=callback_function, options={'disp': True})
# guardo res en una variable para cargar en el futuro
seg = time.time() - seg
np.save('res', res)

c_optimal = res.x.reshape((n_rows, n_cols))
# redondea a 4 decimales c_optimal
c_optimal = np.round(c_optimal, 4)
print(c_optimal)



coeficientes_df = pd.DataFrame(c_optimal, columns=[f'c{j}' for j in range(1, n_cols + 1)])
coeficientes_df.to_csv('coeficientes_optimos.csv', index=False)

print("---------------------------------------------------------")
print("Tiempo de ejecución:", seg, " segundos")
print("Desperdicio inicial con coeficientes equitativos:", objective_function(initial_guess).round(2))
print("Desperdicio final con minimize:", objective_function(c_optimal).round(2))



Iteración: 0
Variables de decisión: [0.3004739  0.39405053 0.09626304 ... 0.31202909 0.07550915 0.07550915]
Valor de la función objetivo: 90.77486425229549
--------------------------
Iteración: 1
Variables de decisión: [0.41135411 0.59796044 0.20206693 ... 0.32980022 0.22418629 0.0855716 ]
Valor de la función objetivo: 100.58297002400427
--------------------------
Iteración: 2
Variables de decisión: [0.39730617 0.61571893 0.31444678 ... 0.32331302 0.3234674  0.07201193]
Valor de la función objetivo: 100.11923655544913
--------------------------
Optimization terminated successfully    (Exit mode 0)
            Current function value: 53.9153637672532
            Iterations: 3
            Function evaluations: 5473
            Gradient evaluations: 3
[[0.3493 0.4839 0.1429 ... 0.3428 0.1639 0.1029]
 [0.3532 0.1395 0.5573 ... 0.     0.5365 0.7198]
 [0.1575 0.197  0.1587 ... 0.3374 0.1585 0.0974]
 [0.14   0.1795 0.1412 ... 0.3199 0.141  0.0799]]
--------------------------------------------

Ahora metodo custom, se limita a optimizar por hora sin compartir la informacion

In [5]:
# Usando metodo custom sin ordenar:
def filaSuma(coeficientes, fila):
    """
    Función para comprobar que la suma de cada fila sea la correspondiente, es decir, n_cols/n_rows. Esta funcion evita que se supere esta cantidad, segunda restriccion.
    Args:
     - coeficientes: DataFrame con los coeficientes de consumo de energía de los usuarios, donde las filas son los usuarios y las columnas son las horas del día durante un mes, array de 4*720 elementos.
    
    Returns:
    - True si la suma de cada fila es 1, False en caso contrario.
     
    """
    if np.sum(coeficientes.iloc[fila, :]) < n_cols/n_rows :
        return False
    return True

def optimization_function(coef, total_energy, consumed_energy):
    """
    Función de optimización para minimizar el desperdicio de energía a la red en una hora.
    
    Args:
    - coef: Coeficientes de reparto de energía para cada usuario.
    - total_energy: Energía total generada por el sistema de placas solares.
    - consumed_energy: Energía consumida por cada usuario.
    
    Returns:
    - waste: Desperdicio total de energía a la red.
    """

    if np.abs(np.sum(coef) - 1) > 1e-6:
        return np.inf
    
    assigned_energy = coef * total_energy
    waste = np.maximum(0, assigned_energy - consumed_energy)
    total_waste = np.sum(waste)
    
    return total_waste


def optimize_energy_distribution(total_energy, consumed_energy):
    """
    Optimiza la distribución de energía entre los usuarios para minimizar el desperdicio de energía a la red en una hora en concreto.
    
    Args:
    - total_energy: Energía total generada por el sistema de placas solares.
    - consumed_energy: Energía consumida por cada usuario.
    
    Returns:
    - optimized_coef: Coeficientes de reparto de energía óptimos.
    - min_waste: Desperdicio mínimo de energía a la red.
    """

    n_users = len(consumed_energy)
    initial_coef = np.ones(n_users) / n_users
    
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                   {'type': 'ineq', 'fun': lambda x: x}, 
                   {'type': 'ineq', 'fun': lambda x: x*total_energy - consumed_energy})
    
    constraints2 = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
               {'type': 'ineq', 'fun': lambda x: x})
    
    result = minimize(optimization_function, initial_coef, args=(total_energy, consumed_energy),
                      constraints=constraints, bounds=[(0, 1)] * n_users)
    
    optimized_coef = result.x
    min_waste = result.fun
    
    return optimized_coef, min_waste


def metodo_consumo_mayor_minimizado(E, total_e):
    """
    Algortimo para optimizar coeficientes de consumo.
    La idea es comenzar por la columna 1 hasta la ultima en orden, tomamos los coeficientes optimos de cada columna. Antes de todo 
    se ha de comprobar que se puede escribir en la celda, 
    que la suma de la fila sea menor que el limit de su fila. Si ya no se puede escribir, ese coeficiente sera 0, y se optimizara con el resto.
    Args:
     - E: DataFrame con los datos de consumo de energía de los usuarios, donde las filas son los usuarios y las columnas son las horas del día durante un mes, array de 4*720 elementos.
     - total_e: Energía total generada por el sistema de placas solares durante un mes por hora, array de 24*30 elementos.
    
    Returns:
    - coeficientes_df: Coeficientes de reparto de energía para cada usuario, tabla de 4x720 elementos.
     
    """
    seg = time.time()
    coeficientes_df = pd.DataFrame(np.full((4, nTotHour), 0), columns=[f'c{j}' for j in range(1, nTotHour + 1)])
    ac = 0
    for j in range(n_cols):
        consumo = [0, 0, 0, 0]
        for i in range(n_rows):
            if filaSuma(coeficientes_df, i)==False:
                consumo[i] = E.iloc[i, j]
            else:
                consumo[i] = 0

        if np.count_nonzero(consumo) == 1:
            coeficientes_df.iloc[np.nonzero(consumo)[0][0], j] = 1  # solo hay un valor distinto de 0, por lo que se pone a 1
        elif np.count_nonzero(consumo) == 2:    # si hay dos valores distintos de 0, se pone a 0.5 (APROXIMACION)
            for i in range(4):
                if consumo[i] != 0:
                    coeficientes_df.iloc[i, j] = 0.5

        else:
            coeficientes, min_waste = optimize_energy_distribution(total_e[j], consumo) # si todos son no nulos, se optimiza
            coeficientes_df.iloc[:, j] = coeficientes
    
    for i in range(4):
        print("La suma de la fila ", i, " es ", np.sum(coeficientes_df.iloc[i, :]), " el porcentaje de aleja de", nTotHour/4,"es ", abs(nTotHour/4-np.sum(coeficientes_df.iloc[i, :]))/180*100, "%")            
    
    return coeficientes_df


resultdf = metodo_consumo_mayor_minimizado(E, total_e)
seg = time.time() - seg

print("---------------------------------------------------------")
print("Tiempo de ejecución:", seg, " segundos")
print("Desperdicio inicial con coeficientes equitativos:", objective_function(initial_guess).round(2))
print("Desperdicio inicial optimizado con metodo_consumo_mayor_minimizado: " , objective_function(resultdf.to_numpy()).round(2))

La suma de la fila  0  es  115.75764844666577  el porcentaje de aleja de 113.75 es  1.1153602481476494 %
La suma de la fila  1  es  111.45190346012569  el porcentaje de aleja de 113.75 es  1.276720299930171 %
La suma de la fila  2  es  113.81282935195208  el porcentaje de aleja de 113.75 es  0.03490519552893071 %
La suma de la fila  3  es  113.9776195791406  el porcentaje de aleja de 113.75 es  0.12645532174477442 %
---------------------------------------------------------
Tiempo de ejecución: 1710931132.9729443  segundos
Desperdicio inicial con coeficientes equitativos: 184.04
Desperdicio inicial optimizado con metodo_consumo_mayor_minimizado:  177.95


Ahora metodo custom que ordena los coeficientes de mayor desperdicio a menor

In [7]:
def calcula_suma_vertido_energia(total_e, E):
    suma_vertido_energia = np.zeros_like(total_e)

    for j in range(nTotHour):
        suma = 0
        generacion = total_e[j]
        for i in range(4):
            consumo = E[i, j]
            if consumo < generacion*0.25:
                suma += generacion*0.25 - consumo
        suma_vertido_energia[j] = suma
    return suma_vertido_energia


def metodo_consumo_mayor_ordenado_minimizado(E, total_e):
    """
    Algortimo para optimizar coeficientes de consumo.
    La idea es comenzar por la columna 1 hasta la 720 en orden, tomamos los coeficientes optimos de cada columna. Antes de todo 
    se ha de comprobar que se puede escribir en la celda, 
    que la suma de la fila sea menor que 180.
    Args:
     - E: DataFrame con los datos de consumo de energía de los usuarios, donde las filas son los usuarios y las columnas son las horas del día durante un mes, array de 4*720 elementos.
     - total_e: Energía total generada por el sistema de placas solares durante un mes por hora, array de 24*30 elementos.
    
    Returns:
    - coeficientes_df: Coeficientes de reparto de energía para cada usuario, tabla de 4x720 elementos.
     
    """
    seg = time.time()
    suma_vertido_energia = calcula_suma_vertido_energia(total_e, E)
    np.savetxt('suma_vertido_energia.txt', suma_vertido_energia, delimiter=',')    
    orden_columnas = np.argsort(suma_vertido_energia)

    orden_columnas = orden_columnas[::-1]    
    E_ordenado = pd.DataFrame(E[:, orden_columnas])  
    
    coeficientes_df_ordenado = metodo_consumo_mayor_minimizado(E_ordenado, total_e)

    coeficientes_df_ordenado = coeficientes_df_ordenado.iloc[:, np.argsort(orden_columnas)]

    return coeficientes_df_ordenado



resultdf = metodo_consumo_mayor_ordenado_minimizado(E_no_nulo.to_numpy(), total_e_no_nulo)
seg = time.time() - seg
# ordena el nombre de la columna, c1 a c...
resultdf.columns = [f'c{j}' for j in range(1, n_cols + 1)]
resultdf.to_csv('coeficientes_optimos_ordenados_custom.csv', index=False)

print("---------------------------------------------------------")
print("Tiempo de ejecución:", seg, " segundos")
print("Desperdicio inicial con coeficientes equitativos:", objective_function(initial_guess).round(2))
print("Desperdicio inicial optimizado con metodo_consumo_mayor_ordenado_minimizado :" , objective_function(resultdf.to_numpy()).round(2))


La suma de la fila  0  es  113.84644753945426  el porcentaje de aleja de 113.75 es  0.05358196636347796 %
La suma de la fila  1  es  112.9066444432415  el porcentaje de aleja de 113.75 es  0.4685308648658311 %
La suma de la fila  2  es  114.22935498436553  el porcentaje de aleja de 113.75 es  0.2663083246475158 %
La suma de la fila  3  es  114.01755306069984  el porcentaje de aleja de 113.75 es  0.1486405892776885 %
---------------------------------------------------------
Tiempo de ejecución: 340.7916660308838  segundos
Desperdicio inicial con coeficientes equitativos: 184.04
Desperdicio inicial optimizado con metodo_consumo_mayor_ordenado_minimizado : 111.85


Los resultados obtenidos son:
|                          | Vertido |
|--------------------------|---------|
| Coeficientes iguales     | 184.04  |
| SLSQP                    | 53.91   |
| Optimiza Horas sin Orden | 177.95  |
| Optimiza Horas Orden     | 111.85  |

Se puede observar que el mejor método es SLSQP, sin embargo, es el mas lento de todos. El tercer metodo propuesto mejora notablemente el vertido de energia a la red con un costo computacional bajo en comparacion con el metodo SLSQP.